In [16]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.applications.resnet50 import preprocess_input
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cv2

In [17]:
def preprocess_old(image_raw):
    image = tf.image.decode_image(image_raw)

    # Prétraitement de l'image : on utilise directement la fonction de ResNet50
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.resnet50.preprocess_input(image)
    image = image[None, ...]
    return image

In [18]:
def preprocess_image(image_raw):
    image_raw = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
    image_raw = cv2.resize(image_raw, (224, 224))
    image_raw = np.expand_dims(image_raw, axis=0)
    
    return image_raw

In [72]:
def generate_targeted_adversaries(model, baseImage, delta, classIdx, target, steps=500):
    loss_object = tf.keras.losses.CategoricalCrossentropy()
    LR = 5e-3
    EPS = 1.0 / 255
    
    previous_image_class = ""
    previous_class_confidence = 0
    
    # Initialize optimizer and loss function
    optimizer = Adam(learning_rate=LR)
    sccLoss = SparseCategoricalCrossentropy()

    # iterate over the number of steps
    for step in range(0, steps):
        # record our gradients
        with tf.GradientTape() as tape:
            # explicitly indicate that our perturbation vector should
            # be tracked for gradient updates
            tape.watch(delta)
            # add our perturbation vector to the base image and
            # preprocess the resulting image
            image = tf.cast(baseImage + delta, tf.float32)
            adversary = preprocess_input(image)
            
            # run this newly constructed image tensor through our
            # model and calculate the loss with respect to the
            # both the *original* class label and the *target*
            # class label
            predictions = model(adversary, training=False)
            
            #image_probs = model.predict(adversary)
            _, image_class, class_confidence = tf.keras.applications.resnet50.decode_predictions(np.array(predictions), top=1)[0][0]
            
            if(previous_image_class != image_class or previous_class_confidence != int(class_confidence)):
                print(image_class + " " + str(class_confidence*100) + "%")
                
            previous_image_class = image_class
            previous_class_confidence = int(class_confidence)
            
            targetLoss = sccLoss(tf.convert_to_tensor([target]), predictions)
            totalLoss = targetLoss#originalLoss + targetLoss
            # check to see if we are logging the loss value, and if
            # so, display it to our terminal
            if step % 20 == 0:
                print("step: {}, totalLoss: {} , targetLoss: {}".format(step, totalLoss.numpy(), targetLoss.numpy()))
        # calculate the gradients of loss with respect to the
        # perturbation vector
        gradients = tape.gradient(totalLoss, delta)
        # update the weights, clip the perturbation vector, and
        # update its value
        optimizer.apply_gradients([(gradients, delta)])
        delta.assign_add(tf.clip_by_value(delta, clip_value_min=-EPS, clip_value_max=EPS))
        
    return delta

In [22]:
def distance(a, b):
    return np.linalg.norm(np.array(a) - np.array(b))

In [23]:
def save_image(image):
    restored = image.numpy().squeeze()
    restored = np.clip(restored, 0, 255).astype("uint8")
    restored = cv2.cvtColor(restored, cv2.COLOR_RGB2BGR)
    cv2.imwrite("adversarial.jpg", restored)

In [65]:
model = tf.keras.applications.resnet50.ResNet50(weights='imagenet')

In [73]:
# Read image
image_raw = cv2.imread("chat.jpg")
image = preprocess_image(image_raw)

In [74]:
baseImage = tf.constant(image, dtype=tf.float32)
delta = tf.Variable(tf.zeros_like(baseImage), trainable=True)

# Generate perturbation vector
deltaUpdated = generate_targeted_adversaries(model, baseImage, delta, 794, 849)

golden_retriever 24.593152105808258%
step: 0, totalLoss: 11.45362663269043 , targetLoss: 11.45362663269043
Persian_cat 20.5671489238739%
step: 20, totalLoss: 8.634535789489746 , targetLoss: 8.634535789489746
step: 40, totalLoss: 6.5191545486450195 , targetLoss: 6.5191545486450195
bittern 7.576622813940048%
step: 60, totalLoss: 4.391037464141846 , targetLoss: 4.391037464141846
coffeepot 6.370828300714493%
teapot 11.032266169786453%
step: 80, totalLoss: 2.106106758117676 , targetLoss: 2.106106758117676
step: 100, totalLoss: 0.6692129373550415 , targetLoss: 0.6692129373550415
step: 120, totalLoss: 0.22752180695533752 , targetLoss: 0.22752180695533752
step: 140, totalLoss: 0.12037176638841629 , targetLoss: 0.12037176638841629
step: 160, totalLoss: 0.08258318156003952 , targetLoss: 0.08258318156003952
step: 180, totalLoss: 0.063374824821949 , targetLoss: 0.063374824821949
step: 200, totalLoss: 0.051507800817489624 , targetLoss: 0.051507800817489624
step: 220, totalLoss: 0.04492322355508804 

In [76]:
# Apply perturbation and save the image
adverImage = (image + deltaUpdated)
save_image(adverImage)

In [77]:
image_probs = model.predict(adverImage)
_, image_class, class_confidence = tf.keras.applications.resnet50.decode_predictions(image_probs, top=1)[0][0]

In [79]:
print("Found: "+ image_class + " with " + str(class_confidence*100) + "% confidence")

Found: saltshaker with 63.78558278083801% confidence
